## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

Изменяем значения по заданию

In [ ]:
DATA_URL = "http://az.lib.ru/t/turgenew_i_s/text_0070.shtml" # Ссылка на файл из Задания
more_words_count = 100
tokens_more_count = 20

Устанавливаем библиотеки

In [ ]:
! pip install -q PyYaml==5.3.1
! pip install -q rnnmorph==0.4.0
! pip install -q nltk==3.2.5
! pip install 'h5py==2.10.0' -force-reinstall

     |████████████████████████████████| 269 kB 14.4 MB/s 
     |████████████████████████████████| 10.5 MB 12.8 MB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 60.8 MB/s 
Looking in links: orce-reinstall
     |████████████████████████████████| 2.9 MB 13.8 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


Запускаем код и смотрим внизу ответы

In [ ]:
%tensorflow_version 1.x
import nltk
import warnings
import urllib.request
from tqdm import tqdm
from nltk import FreqDist
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
warnings.filterwarnings('ignore')
nltk.download('punkt', quiet=True)
nltk.download("stopwords", quiet=True)

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")
opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset())

soup = BeautifulSoup(raw_text, features="html.parser")

for script in soup(["script", "style"]):
    script.extract()

cleaned_text = soup.get_text()
tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()] for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ] # ИСПРАВИЛ НА ТОЛЬКО БУКВЫ
non_uniq_tokens = [word for sentence in predictions for word in sentence]

answers = {}
for i in non_uniq_tokens:
  if i in answers:
    answers[i] += 1
  else:
    answers[i] = 1

STOPWORDS = set(stopwords.words("russian"))

t = 1
counter = more_words_count + 1
while counter > more_words_count:
  counter = 0
  n_answers = []
  for i in answers:
    if (answers[i] > t):
      n_answers.append(i)
      counter += 1
  t += 1

counter = 0
for i in n_answers:
  if i in stopwords.words("russian"):
    counter += 1

counter1 = 0
for i in answers:
  if (answers[i] > tokens_more_count):
    counter1 += 1
print("\n")

print("Введите количество предложений")
print("Ответ:", len(predictions))

print("Введите количество токенов, состоящих только из букв")
print("Ответ:", len(non_uniq_tokens))

print(f"Какую долю среди {more_words_count} самых частотных токенов в произведении занимают слова, не входящие в стоп-лист?")
print("Ответ:", (more_words_count - counter) / more_words_count)

print(f"Сколько токенов встречается в тексте строго больше {tokens_more_count} раз?")
print("Ответ:", counter1)

sentences: 100%|██████████| 532/532 [00:00<00:00, 87786.99it/s]



Введите количество предложений
Ответ: 532
Введите количество токенов, состоящих только из букв
Ответ: 8354
Какую долю среди 100 самых частотных токенов в произведении занимают слова, не входящие в стоп-лист?
Ответ: 0.5
Сколько токенов встречается в тексте строго больше 20 раз?
Ответ: 52
